In [75]:
!pip install datasets
!pip install tiktoken

!pip install --upgrade transformers
!pip install sentencepiece

!pip install rouge_score
!pip install evaluate

!pip install nltk

!pip install accelerate

In [1]:
from datasets import load_dataset
from datasets import Dataset
from datasets import concatenate_datasets, DatasetDict
import pandas as pd
from transformers import AutoTokenizer
from rouge_score import rouge_scorer
import evaluate 
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoModelForSeq2SeqLM

from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq

from torch.optim import AdamW

from accelerate import Accelerator
from transformers import get_scheduler

from tqdm.auto import tqdm
import torch
import numpy as np

C:\Users\xavie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#ds_fr = load_dataset("mteb/amazon_reviews_multi", "fr")
#ds_en = load_dataset("SetFit/amazon_reviews_multi_en")
#print(ds_fr)
#print(ds_fr["train"][0])

#DATASET SUCKS SO WE DOWNLODED CSV FILES AND CONVERTED TO DATASET INSTEAD

In [3]:
df_train = pd.read_csv(r"C:\Users\xavie\OneDrive\Bureau\Code\Data\data_summarize\train.csv")
df_test = pd.read_csv(r"C:\Users\xavie\OneDrive\Bureau\Code\Data\data_summarize\test.csv")
df_val = pd.read_csv(r"C:\Users\xavie\OneDrive\Bureau\Code\Data\data_summarize\validation.csv")

In [4]:
print(df_train.columns)  # should include review_body, review_title, label, etc.
print(df_train.head(2))

Index(['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars',
       'review_body', 'review_title', 'language', 'product_category'],
      dtype='object')
   Unnamed: 0   review_id          product_id          reviewer_id  stars  \
0           0  de_0203609  product_de_0865382  reviewer_de_0267719      1   
1           1  de_0559494  product_de_0678997  reviewer_de_0783625      1   

                                      review_body               review_title  \
0  Armband ist leider nach 1 Jahr kaputt gegangen  Leider nach 1 Jahr kaputt   
1              In der Lieferung war nur Ein Akku!   EINS statt ZWEI Akkus!!!   

  language  product_category  
0       de            sports  
1       de  home_improvement  


In [5]:
cond_train_fr = df_train.language == "fr"
cond_train_en = df_train.language == "en"
cond_test_fr = df_test.language == "fr"
cond_test_en = df_test.language == "en"
cond_val_fr = df_val.language == "fr"
cond_val_en = df_val.language == "en"


dft_fr = df_train[cond_train_fr] # train data french
dft_en = df_train[cond_train_en] # train data english
dfs_fr = df_test[cond_test_fr]   # test data french
dfs_en = df_test[cond_test_en]   # test data english
dfv_fr = df_val[cond_val_fr]     # validation data french
dfv_en = df_val[cond_val_en]     # validation data english

In [6]:
# Show counts for top 20 products
dft_en["product_category"].value_counts()[:20]

product_category
home                      17679
apparel                   15951
wireless                  15717
other                     13418
beauty                    12091
drugstore                 11730
kitchen                   10382
toy                        8745
sports                     8277
automotive                 7506
lawn_and_garden            7327
home_improvement           7136
pet_products               7082
digital_ebook_purchase     6749
pc                         6401
electronics                6186
office_product             5521
shoes                      5197
grocery                    4730
book                       3756
Name: count, dtype: int64

In [7]:
ds_en = Dataset.from_pandas(dft_en)
ds_fr = Dataset.from_pandas(dft_fr)
ds_en_valid = Dataset.from_pandas(dfv_en)
ds_fr_valid = Dataset.from_pandas(dfv_fr)
ds_en_test = Dataset.from_pandas(dfs_en)
ds_fr_test = Dataset.from_pandas(dfs_fr)

In [8]:
print(ds_en)

Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', '__index_level_0__'],
    num_rows: 200000
})


In [9]:
features_list = list(ds_en.features.keys()) ; features_list # returns dict_keys object

['Unnamed: 0',
 'review_id',
 'product_id',
 'reviewer_id',
 'stars',
 'review_body',
 'review_title',
 'language',
 'product_category',
 '__index_level_0__']

In [10]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset.shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


In [11]:
show_samples(ds_en)


'>> Title: Worked in front position, not rear'
'>> Review: 3 stars because these are not rear brakes as stated in the item description. At least the mount adapter only worked on the front fork of the bike that I got it for.'

'>> Title: meh'
'>> Review: Does it’s job and it’s gorgeous but mine is falling apart, I had to basically put it together again with hot glue'

'>> Title: Can't beat these for the money'
'>> Review: Bought this for handling miscellaneous aircraft parts and hanger "stuff" that I needed to organize; it really fit the bill. The unit arrived quickly, was well packaged and arrived intact (always a good sign). There are five wall mounts-- three on the top and two on the bottom. I wanted to mount it on the wall, so all I had to do was to remove the top two layers of plastic drawers, as well as the bottom corner drawers, place it when I wanted and mark it; I then used some of the new plastic screw in wall anchors (the 50 pound variety) and it easily mounted to the wall. 

In [12]:
def filter_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )



In [13]:
french_books = ds_fr.filter(filter_books)
english_books = ds_en.filter(filter_books)
french_books_valid = ds_fr_valid.filter(filter_books)
english_books_valid = ds_en_valid.filter(filter_books)
french_books_test = ds_fr_test.filter(filter_books)
english_books_test = ds_en_test.filter(filter_books)

show_samples(english_books)

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter: 100%|██████████| 5000/5000 [00:00<00:00, 61734.14 examples/s]


'>> Title: I'm dissapointed.'
'>> Review: I guess I had higher expectations for this book from the reviews. I really thought I'd at least like it. The plot idea was great. I loved Ash but, it just didnt go anywhere. Most of the book was about their radio show and talking to callers. I wanted the author to dig deeper so we could really get to know the characters. All we know about Grace is that she is attractive looking, Latino and is kind of a brat. I'm dissapointed.'

'>> Title: Good art, good price, poor design'
'>> Review: I had gotten the DC Vintage calendar the past two years, but it was on backorder forever this year and I saw they had shrunk the dimensions for no good reason. This one has good art choices but the design has the fold going through the picture, so it's less aesthetically pleasing, especially if you want to keep a picture to hang. For the price, a good calendar'

'>> Title: Helpful'
'>> Review: Nearly all the tips useful and. I consider myself an intermediate to a

In [14]:
english_books

Dataset({
    features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', '__index_level_0__'],
    num_rows: 10505
})

In [15]:
books_dataset = DatasetDict()
books_valid = DatasetDict()
books_test = DatasetDict()

for split in features_list:
    books_dataset = concatenate_datasets(
        [english_books, french_books]
    )
    books_valid = concatenate_datasets(
        [english_books_valid, french_books_valid]
    )
    books_test = concatenate_datasets(
        [english_books_test, french_books_test]
    )
    books_dataset = books_dataset.shuffle(seed=42)
    books_valid = books_valid.shuffle(seed=42)
    books_test = books_test.shuffle(seed=42)

# Peek at a few examples
show_samples(books_dataset)


'>> Title: Love this book'
'>> Review: Power in the tongue we speak life or death and I t is biblical Great read Biblical principles we can live by'

'>> Title: Satisfaite'
'>> Review: Conforme à la description. Je recommande.'

'>> Title: This book is a classic'
'>> Review: Classic book, great read! Theoretical but also practical. Author of writes about fragment exercises you can do which I found interesting'


In [16]:
# Filter the books_dataset "train" split
# Filter keeping only non-empty titles longer than 2 characters
books_dataset = books_dataset.filter(
    lambda example: example["review_title"] is not None and len(example["review_title"]) > 2
)

books_valid = books_valid.filter(
    lambda example: example["review_title"] is not None and len(example["review_title"]) > 2
)

books_test = books_test.filter(
    lambda example: example["review_title"] is not None and len(example["review_title"]) > 2
)

Filter:   5%|▍         | 1000/20500 [00:00<00:02, 9181.18 examples/s]

Filter: 100%|██████████| 532/532 [00:00<00:00, 12077.26 examples/s]


In [17]:
tokenizer = AutoTokenizer.from_pretrained("google/byt5-small")

In [18]:
inputs = tokenizer("I loved reading the Hunger Games!")
inputs

{'input_ids': [76, 35, 111, 114, 121, 104, 103, 35, 117, 104, 100, 103, 108, 113, 106, 35, 119, 107, 104, 35, 75, 120, 113, 106, 104, 117, 35, 74, 100, 112, 104, 118, 36, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
tokenizer.convert_ids_to_tokens(inputs.input_ids)

['I',
 ' ',
 'l',
 'o',
 'v',
 'e',
 'd',
 ' ',
 'r',
 'e',
 'a',
 'd',
 'i',
 'n',
 'g',
 ' ',
 't',
 'h',
 'e',
 ' ',
 'H',
 'u',
 'n',
 'g',
 'e',
 'r',
 ' ',
 'G',
 'a',
 'm',
 'e',
 's',
 '!',
 '</s>']

In [20]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["review_body"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["review_title"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [21]:
tokenized_datasets = books_dataset.map(preprocess_function, batched=True)
tokenized_valid = books_valid.map(preprocess_function, batched=True)

Map:   0%|          | 0/20387 [00:00<?, ? examples/s]

Map: 100%|██████████| 495/495 [00:00<00:00, 2827.66 examples/s]


In [22]:
rouge_score = evaluate.load("rouge")

In [23]:
generated_summary = "I absolutely loved reading the Hunger Games"
reference_summary = "I loved reading the Hunger Games"

scores = rouge_score.compute(
    predictions=[generated_summary], references=[reference_summary]
)
scores

{'rouge1': np.float64(0.923076923076923),
 'rouge2': np.float64(0.7272727272727272),
 'rougeL': np.float64(0.923076923076923),
 'rougeLsum': np.float64(0.923076923076923)}

In [24]:
# Download the full set of Punkt tokenizers
nltk.download('punkt')       # English
nltk.download('punkt_tab')   # Needed for some multilingual tokenization

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\xavie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [25]:
# Your function
def three_sentence_summary(text, lang="english"):
    return "\n".join(sent_tokenize(text, language=lang)[:3])

In [26]:
three_sentence_summary(books_dataset["review_body"][0])

'This was a fantastically entertaining book that I couldn’t put down.\nThe characters were great and I cried for about 50% of th book.\nI would have given it a higher rating, but just have a feeling it won’t stick with me long-term.'

In [27]:
def evaluate_baseline(dataset, metric):
    summaries = [three_sentence_summary(text) for text in dataset["review_body"]]
    return metric.compute(predictions=summaries, references=dataset["review_title"])

In [28]:
score = evaluate_baseline(books_dataset, rouge_score)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, round(score[rn] * 100, 2)) for rn in rouge_names)
rouge_dict

{'rouge1': np.float64(10.96),
 'rouge2': np.float64(5.12),
 'rougeL': np.float64(10.31),
 'rougeLsum': np.float64(10.53)}

In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/byt5-small")


In [30]:
tokenized_datasets = tokenized_datasets.remove_columns(
    books_dataset.column_names
)





In [31]:
tokenized_valid = tokenized_valid.remove_columns(
    books_valid.column_names
)

In [32]:
tokenized_datasets.set_format("torch")
tokenized_valid.set_format("torch")

In [33]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [34]:
batch_size = 8
train_dataloader = DataLoader(
    tokenized_datasets,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)

eval_dataloader = DataLoader(
    tokenized_valid, collate_fn=data_collator, batch_size=batch_size
)

In [35]:
optimizer = AdamW(model.parameters(), lr=2e-5)

In [36]:
accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [37]:
num_train_epochs = 10
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [38]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # ROUGE expects a newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [83]:
progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result)

    # Save model After all epochs are done
unwrapped_model = accelerator.unwrap_model(model)
output_dir = "./fine_tuned_model"

unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
tokenizer.save_pretrained(output_dir)


  0%|          | 0/25490 [00:00<?, ?it/s]C:\Users\xavie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\data\data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)
  0%|          | 4/25490 [09:04<1027:00:02, 145.07s/it]

KeyboardInterrupt: 

In [39]:
tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_model")
summarizer = AutoModelForSeq2SeqLM.from_pretrained("./fine_tuned_model")

HFValidationError: Repo id must use alphanumeric chars, '-', '_' or '.'. The name cannot start or end with '-' or '.' and the maximum length is 96: './fine_tuned_model'.

In [40]:
def print_summary(idx):
    review = books_test[idx]["review_body"]
    title = books_test[idx]["review_title"]
    summary = summarizer(books_test[idx]["review_body"])[0]["summary_text"]
    print(f"'>>> Review: {review}'")
    print(f"\n'>>> Title: {title}'")
    print(f"\n'>>> Summary: {summary}'")

In [41]:
print_summary(100)


NameError: name 'summarizer' is not defined

In [ ]:
print_summary(0)